In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
columns = pd.read_csv("/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv").columns

In [ ]:
!ls ../input/openmmlab-offline-installation-mmcv-full/

In [ ]:
!pip install /kaggle/input/mmcv161py3/mmcv-1.6.1-py2.py3-none-any.whl -f ./ --no-index --no-deps

In [ ]:
!pip install /kaggle/input/pydicom-offline-installer/pydicom-2.3.1-py3-none-any.whl -f ./ --no-index --no-deps

In [ ]:
!pip install /kaggle/input/simpleitk-offline-installation-wheel/SimpleITK-2.3.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index --no-deps

In [ ]:
!pip install /kaggle/input/simpleitk-offline-installation-wheel/addict-2.4.0-py3-none-any.whl -f ./ --no-index --no-deps

In [ ]:
!cp -r /kaggle/input/yolov8/ultralytics /kaggle/working/

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import cv2
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import ModelCheckpoint
from typing import Tuple, List
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import SimpleITK as sitk
# from datasets.seg_dataset import DatasetSegTest
# from aug.seg_aug import SameResAugsAlbu
# from losses.seg_loss import SegFastLossCalculator
# from metrics.seg_metrics import multiclass_dice_coefficient
import glob

from ultralytics import YOLO
import joblib

from imblearn.ensemble import BalancedRandomForestClassifier

In [ ]:
import logging
import warnings

import torch
from mmcv.cnn import build_activation_layer, ConvModule, NonLocal3d#, kaiming_init, constant_init
#from mmcv.runner import _load_checkpoint, load_checkpoint
from torch import nn
from torch.nn.modules.batchnorm import _BatchNorm
from torch.nn.modules.utils import _ntuple, _triple


class BasicBlock3d(nn.Module):
    """BasicBlock 3d block for ResNet3D.
    Args:
        inplanes (int): Number of channels for the input in first conv3d layer.
        planes (int): Number of channels produced by some norm/conv3d layers.
        spatial_stride (int): Spatial stride in the conv3d layer. Default: 1.
        temporal_stride (int): Temporal stride in the conv3d layer. Default: 1.
        dilation (int): Spacing between kernel elements. Default: 1.
        downsample (nn.Module | None): Downsample layer. Default: None.
        style (str): ``pytorch`` or ``caffe``. If set to "pytorch", the
            stride-two layer is the 3x3 conv layer, otherwise the stride-two
            layer is the first 1x1 conv layer. Default: 'pytorch'.
        inflate (bool): Whether to inflate kernel. Default: True.
        non_local (bool): Determine whether to apply non-local module in this
            block. Default: False.
        non_local_cfg (dict): Config for non-local module. Default: ``dict()``.
        conv_cfg (dict): Config dict for convolution layer.
            Default: ``dict(type='Conv3d')``.
        norm_cfg (dict): Config for norm layers. required keys are ``type``,
            Default: ``dict(type='BN3d')``.
        act_cfg (dict): Config dict for activation layer.
            Default: ``dict(type='ReLU')``.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed. Default: False.
    """
    expansion = 1

    def __init__(self,
                 inplanes,
                 planes,
                 spatial_stride=1,
                 temporal_stride=1,
                 dilation=1,
                 downsample=None,
                 style='pytorch',
                 inflate=True,
                 non_local=False,
                 non_local_cfg=dict(),
                 conv_cfg=dict(type='Conv3d'),
                 norm_cfg=dict(type='BN3d'),
                 act_cfg=dict(type='ReLU'),
                 with_cp=False,
                 **kwargs):
        super().__init__()
        assert style in ['pytorch', 'caffe']
        # make sure that only ``inflate_style`` is passed into kwargs
        assert set(kwargs).issubset(['inflate_style'])

        self.inplanes = inplanes
        self.planes = planes
        self.spatial_stride = spatial_stride
        self.temporal_stride = temporal_stride
        self.dilation = dilation
        self.style = style
        self.inflate = inflate
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.act_cfg = act_cfg
        self.with_cp = with_cp
        self.non_local = non_local
        self.non_local_cfg = non_local_cfg

        self.conv1_stride_s = spatial_stride
        self.conv2_stride_s = 1
        self.conv1_stride_t = temporal_stride
        self.conv2_stride_t = 1

        if self.inflate:
            conv1_kernel_size = (3, 3, 3)
            conv1_padding = (1, dilation, dilation)
            conv2_kernel_size = (3, 3, 3)
            conv2_padding = (1, 1, 1)
        else:
            conv1_kernel_size = (1, 3, 3)
            conv1_padding = (0, dilation, dilation)
            conv2_kernel_size = (1, 3, 3)
            conv2_padding = (0, 1, 1)

        self.conv1 = ConvModule(
            inplanes,
            planes,
            conv1_kernel_size,
            stride=(self.conv1_stride_t, self.conv1_stride_s,
                    self.conv1_stride_s),
            padding=conv1_padding,
            dilation=(1, dilation, dilation),
            bias=False,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            act_cfg=self.act_cfg)

        self.conv2 = ConvModule(
            planes,
            planes * self.expansion,
            conv2_kernel_size,
            stride=(self.conv2_stride_t, self.conv2_stride_s,
                    self.conv2_stride_s),
            padding=conv2_padding,
            bias=False,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            act_cfg=None)

        self.downsample = downsample
        self.relu = build_activation_layer(self.act_cfg)

        if self.non_local:
            self.non_local_block = NonLocal3d(self.conv2.norm.num_features,
                                              **self.non_local_cfg)

    def forward(self, x):
        """Defines the computation performed at every call."""

        def _inner_forward(x):
            """Forward wrapper for utilizing checkpoint."""
            identity = x

            out = self.conv1(x)
            out = self.conv2(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            out = out + identity
            return out

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(_inner_forward, x)
        else:
            out = _inner_forward(x)
        out = self.relu(out)

        if self.non_local:
            out = self.non_local_block(out)

        return out


class Bottleneck3d(nn.Module):
    """Bottleneck 3d block for ResNet3D.
    Args:
        inplanes (int): Number of channels for the input in first conv3d layer.
        planes (int): Number of channels produced by some norm/conv3d layers.
        spatial_stride (int): Spatial stride in the conv3d layer. Default: 1.
        temporal_stride (int): Temporal stride in the conv3d layer. Default: 1.
        dilation (int): Spacing between kernel elements. Default: 1.
        downsample (nn.Module | None): Downsample layer. Default: None.
        style (str): ``pytorch`` or ``caffe``. If set to "pytorch", the
            stride-two layer is the 3x3 conv layer, otherwise the stride-two
            layer is the first 1x1 conv layer. Default: 'pytorch'.
        inflate (bool): Whether to inflate kernel. Default: True.
        inflate_style (str): ``3x1x1`` or ``3x3x3``. which determines the
            kernel sizes and padding strides for conv1 and conv2 in each block.
            Default: '3x1x1'.
        non_local (bool): Determine whether to apply non-local module in this
            block. Default: False.
        non_local_cfg (dict): Config for non-local module. Default: ``dict()``.
        conv_cfg (dict): Config dict for convolution layer.
            Default: ``dict(type='Conv3d')``.
        norm_cfg (dict): Config for norm layers. required keys are ``type``,
            Default: ``dict(type='BN3d')``.
        act_cfg (dict): Config dict for activation layer.
            Default: ``dict(type='ReLU')``.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed. Default: False.
    """
    expansion = 4

    def __init__(self,
                 inplanes,
                 planes,
                 spatial_stride=1,
                 temporal_stride=1,
                 dilation=1,
                 downsample=None,
                 style='pytorch',
                 inflate=True,
                 inflate_style='3x1x1',
                 non_local=False,
                 non_local_cfg=dict(),
                 conv_cfg=dict(type='Conv3d'),
                 norm_cfg=dict(type='BN3d'),
                 act_cfg=dict(type='ReLU'),
                 with_cp=False):
        super().__init__()
        assert style in ['pytorch', 'caffe']
        assert inflate_style in ['3x1x1', '3x3x3']

        self.inplanes = inplanes
        self.planes = planes
        self.spatial_stride = spatial_stride
        self.temporal_stride = temporal_stride
        self.dilation = dilation
        self.style = style
        self.inflate = inflate
        self.inflate_style = inflate_style
        self.norm_cfg = norm_cfg
        self.conv_cfg = conv_cfg
        self.act_cfg = act_cfg
        self.with_cp = with_cp
        self.non_local = non_local
        self.non_local_cfg = non_local_cfg

        if self.style == 'pytorch':
            self.conv1_stride_s = 1
            self.conv2_stride_s = spatial_stride
            self.conv1_stride_t = 1
            self.conv2_stride_t = temporal_stride
        else:
            self.conv1_stride_s = spatial_stride
            self.conv2_stride_s = 1
            self.conv1_stride_t = temporal_stride
            self.conv2_stride_t = 1

        if self.inflate:
            if inflate_style == '3x1x1':
                conv1_kernel_size = (3, 1, 1)
                conv1_padding = (1, 0, 0)
                conv2_kernel_size = (1, 3, 3)
                conv2_padding = (0, dilation, dilation)
            else:
                conv1_kernel_size = (1, 1, 1)
                conv1_padding = (0, 0, 0)
                conv2_kernel_size = (3, 3, 3)
                conv2_padding = (1, dilation, dilation)
        else:
            conv1_kernel_size = (1, 1, 1)
            conv1_padding = (0, 0, 0)
            conv2_kernel_size = (1, 3, 3)
            conv2_padding = (0, dilation, dilation)

        self.conv1 = ConvModule(
            inplanes,
            planes,
            conv1_kernel_size,
            stride=(self.conv1_stride_t, self.conv1_stride_s,
                    self.conv1_stride_s),
            padding=conv1_padding,
            bias=False,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            act_cfg=self.act_cfg)

        self.conv2 = ConvModule(
            planes,
            planes,
            conv2_kernel_size,
            stride=(self.conv2_stride_t, self.conv2_stride_s,
                    self.conv2_stride_s),
            padding=conv2_padding,
            dilation=(1, dilation, dilation),
            bias=False,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            act_cfg=self.act_cfg)

        self.conv3 = ConvModule(
            planes,
            planes * self.expansion,
            1,
            bias=False,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            # No activation in the third ConvModule for bottleneck
            act_cfg=None)

        self.downsample = downsample
        self.relu = build_activation_layer(self.act_cfg)

        if self.non_local:
            self.non_local_block = NonLocal3d(self.conv3.norm.num_features,
                                              **self.non_local_cfg)

    def forward(self, x):
        """Defines the computation performed at every call."""

        def _inner_forward(x):
            """Forward wrapper for utilizing checkpoint."""
            identity = x

            out = self.conv1(x)
            out = self.conv2(out)
            out = self.conv3(out)

            if self.downsample is not None:
                identity = self.downsample(x)

            out = out + identity
            return out

        if self.with_cp and x.requires_grad:
            out = cp.checkpoint(_inner_forward, x)
        else:
            out = _inner_forward(x)
        out = self.relu(out)

        if self.non_local:
            out = self.non_local_block(out)

        return out


class ResNet3d(nn.Module):
    """ResNet 3d backbone.
    Args:
        depth (int): Depth of resnet, from {18, 34, 50, 101, 152}.
        pretrained (str | None): Name of pretrained model.
        stage_blocks (tuple | None): Set number of stages for each res layer.
            Default: None.
        pretrained2d (bool): Whether to load pretrained 2D model.
            Default: True.
        in_channels (int): Channel num of input features. Default: 3.
        base_channels (int): Channel num of stem output features. Default: 64.
        out_indices (Sequence[int]): Indices of output feature. Default: (3, ).
        num_stages (int): Resnet stages. Default: 4.
        spatial_strides (Sequence[int]):
            Spatial strides of residual blocks of each stage.
            Default: ``(1, 2, 2, 2)``.
        temporal_strides (Sequence[int]):
            Temporal strides of residual blocks of each stage.
            Default: ``(1, 1, 1, 1)``.
        dilations (Sequence[int]): Dilation of each stage.
            Default: ``(1, 1, 1, 1)``.
        conv1_kernel (Sequence[int]): Kernel size of the first conv layer.
            Default: ``(3, 7, 7)``.
        conv1_stride_s (int): Spatial stride of the first conv layer.
            Default: 2.
        conv1_stride_t (int): Temporal stride of the first conv layer.
            Default: 1.
        pool1_stride_s (int): Spatial stride of the first pooling layer.
            Default: 2.
        pool1_stride_t (int): Temporal stride of the first pooling layer.
            Default: 1.
        with_pool2 (bool): Whether to use pool2. Default: True.
        style (str): `pytorch` or `caffe`. If set to "pytorch", the stride-two
            layer is the 3x3 conv layer, otherwise the stride-two layer is
            the first 1x1 conv layer. Default: 'pytorch'.
        frozen_stages (int): Stages to be frozen (all param fixed). -1 means
            not freezing any parameters. Default: -1.
        inflate (Sequence[int]): Inflate Dims of each block.
            Default: (1, 1, 1, 1).
        inflate_style (str): ``3x1x1`` or ``3x3x3``. which determines the
            kernel sizes and padding strides for conv1 and conv2 in each block.
            Default: '3x1x1'.
        conv_cfg (dict): Config for conv layers. required keys are ``type``
            Default: ``dict(type='Conv3d')``.
        norm_cfg (dict): Config for norm layers. required keys are ``type`` and
            ``requires_grad``.
            Default: ``dict(type='BN3d', requires_grad=True)``.
        act_cfg (dict): Config dict for activation layer.
            Default: ``dict(type='ReLU', inplace=True)``.
        norm_eval (bool): Whether to set BN layers to eval mode, namely, freeze
            running stats (mean and var). Default: False.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed. Default: False.
        non_local (Sequence[int]): Determine whether to apply non-local module
            in the corresponding block of each stages. Default: (0, 0, 0, 0).
        non_local_cfg (dict): Config for non-local module. Default: ``dict()``.
        zero_init_residual (bool):
            Whether to use zero initialization for residual block,
            Default: True.
        kwargs (dict, optional): Key arguments for "make_res_layer".
    """

    arch_settings = {
        18: (BasicBlock3d, (2, 2, 2, 2)),
        34: (BasicBlock3d, (3, 4, 6, 3)),
        50: (Bottleneck3d, (3, 4, 6, 3)),
        101: (Bottleneck3d, (3, 4, 23, 3)),
        152: (Bottleneck3d, (3, 8, 36, 3))
    }

    def __init__(self,
                 depth,
                 pretrained,
                 stage_blocks=None,
                 pretrained2d=True,
                 in_channels=3,
                 num_stages=4,
                 base_channels=64,
                 out_indices=(0, 1, 2, 3,),
                 spatial_strides=(1, 2, 2, 2),
                 temporal_strides=(1, 1, 1, 1),
                 dilations=(1, 1, 1, 1),
                 conv1_kernel=(3, 7, 7),
                 conv1_stride_s=2,
                 conv1_stride_t=1,
                 pool1_stride_s=2,
                 pool1_stride_t=1,
                 with_pool1=True,
                 with_pool2=True,
                 style='pytorch',
                 frozen_stages=-1,
                 inflate=(1, 1, 1, 1),
                 inflate_style='3x1x1',
                 conv_cfg=dict(type='Conv3d'),
                 norm_cfg=dict(type='BN3d', requires_grad=True),
                 act_cfg=dict(type='ReLU', inplace=True),
                 norm_eval=False,
                 with_cp=False,
                 non_local=(0, 0, 0, 0),
                 non_local_cfg=dict(),
                 zero_init_residual=True,
                 **kwargs):
        super().__init__()
        if depth not in self.arch_settings:
            raise KeyError(f'invalid depth {depth} for resnet')
        self.depth = depth
        self.pretrained = pretrained
        self.pretrained2d = pretrained2d
        self.in_channels = in_channels
        self.base_channels = base_channels
        self.num_stages = num_stages
        assert 1 <= num_stages <= 4
        self.stage_blocks = stage_blocks
        self.out_indices = out_indices
        assert max(out_indices) < num_stages
        self.spatial_strides = spatial_strides
        self.temporal_strides = temporal_strides
        self.dilations = dilations
        assert len(spatial_strides) == len(temporal_strides) == len(
            dilations) == num_stages
        if self.stage_blocks is not None:
            assert len(self.stage_blocks) == num_stages

        self.conv1_kernel = conv1_kernel
        self.conv1_stride_s = conv1_stride_s
        self.conv1_stride_t = conv1_stride_t
        self.pool1_stride_s = pool1_stride_s
        self.pool1_stride_t = pool1_stride_t
        self.with_pool1 = with_pool1
        self.with_pool2 = with_pool2
        self.style = style
        self.frozen_stages = frozen_stages
        self.stage_inflations = _ntuple(num_stages)(inflate)
        self.non_local_stages = _ntuple(num_stages)(non_local)
        self.inflate_style = inflate_style
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.act_cfg = act_cfg
        self.norm_eval = norm_eval
        self.with_cp = with_cp
        self.zero_init_residual = zero_init_residual

        self.block, stage_blocks = self.arch_settings[depth]

        if self.stage_blocks is None:
            self.stage_blocks = stage_blocks[:num_stages]

        self.inplanes = self.base_channels

        self.non_local_cfg = non_local_cfg

        self._make_stem_layer()

        self.res_layers = []
        for i, num_blocks in enumerate(self.stage_blocks):
            spatial_stride = spatial_strides[i]
            temporal_stride = temporal_strides[i]
            dilation = dilations[i]
            planes = self.base_channels * 2 ** i
            res_layer = self.make_res_layer(
                self.block,
                self.inplanes,
                planes,
                num_blocks,
                spatial_stride=spatial_stride,
                temporal_stride=temporal_stride,
                dilation=dilation,
                style=self.style,
                norm_cfg=self.norm_cfg,
                conv_cfg=self.conv_cfg,
                act_cfg=self.act_cfg,
                non_local=self.non_local_stages[i],
                non_local_cfg=self.non_local_cfg,
                inflate=self.stage_inflations[i],
                inflate_style=self.inflate_style,
                with_cp=with_cp,
                **kwargs)
            self.inplanes = planes * self.block.expansion
            layer_name = f'layer{i + 1}'
            self.add_module(layer_name, res_layer)
            self.res_layers.append(layer_name)

        self.feat_dim = self.block.expansion * self.base_channels * 2 ** (
                len(self.stage_blocks) - 1)

    @staticmethod
    def make_res_layer(block,
                       inplanes,
                       planes,
                       blocks,
                       spatial_stride=1,
                       temporal_stride=1,
                       dilation=1,
                       style='pytorch',
                       inflate=1,
                       inflate_style='3x1x1',
                       non_local=0,
                       non_local_cfg=dict(),
                       norm_cfg=None,
                       act_cfg=None,
                       conv_cfg=None,
                       with_cp=False,
                       **kwargs):
        """Build residual layer for ResNet3D.
        Args:
            block (nn.Module): Residual module to be built.
            inplanes (int): Number of channels for the input feature
                in each block.
            planes (int): Number of channels for the output feature
                in each block.
            blocks (int): Number of residual blocks.
            spatial_stride (int | Sequence[int]): Spatial strides in
                residual and conv layers. Default: 1.
            temporal_stride (int | Sequence[int]): Temporal strides in
                residual and conv layers. Default: 1.
            dilation (int): Spacing between kernel elements. Default: 1.
            style (str): ``pytorch`` or ``caffe``. If set to ``pytorch``,
                the stride-two layer is the 3x3 conv layer, otherwise
                the stride-two layer is the first 1x1 conv layer.
                Default: ``pytorch``.
            inflate (int | Sequence[int]): Determine whether to inflate
                for each block. Default: 1.
            inflate_style (str): ``3x1x1`` or ``3x3x3``. which determines
                the kernel sizes and padding strides for conv1 and conv2
                in each block. Default: '3x1x1'.
            non_local (int | Sequence[int]): Determine whether to apply
                non-local module in the corresponding block of each stages.
                Default: 0.
            non_local_cfg (dict): Config for non-local module.
                Default: ``dict()``.
            conv_cfg (dict | None): Config for norm layers. Default: None.
            norm_cfg (dict | None): Config for norm layers. Default: None.
            act_cfg (dict | None): Config for activate layers. Default: None.
            with_cp (bool | None): Use checkpoint or not. Using checkpoint
                will save some memory while slowing down the training speed.
                Default: False.
        Returns:
            nn.Module: A residual layer for the given config.
        """
        inflate = inflate if not isinstance(inflate,
                                            int) else (inflate,) * blocks
        non_local = non_local if not isinstance(
            non_local, int) else (non_local,) * blocks
        assert len(inflate) == blocks and len(non_local) == blocks
        downsample = None
        if spatial_stride != 1 or inplanes != planes * block.expansion:
            downsample = ConvModule(
                inplanes,
                planes * block.expansion,
                kernel_size=1,
                stride=(temporal_stride, spatial_stride, spatial_stride),
                bias=False,
                conv_cfg=conv_cfg,
                norm_cfg=norm_cfg,
                act_cfg=None)

        layers = []
        layers.append(
            block(
                inplanes,
                planes,
                spatial_stride=spatial_stride,
                temporal_stride=temporal_stride,
                dilation=dilation,
                downsample=downsample,
                style=style,
                inflate=(inflate[0] == 1),
                inflate_style=inflate_style,
                non_local=(non_local[0] == 1),
                non_local_cfg=non_local_cfg,
                norm_cfg=norm_cfg,
                conv_cfg=conv_cfg,
                act_cfg=act_cfg,
                with_cp=with_cp,
                **kwargs))
        inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(
                block(
                    inplanes,
                    planes,
                    spatial_stride=1,
                    temporal_stride=1,
                    dilation=dilation,
                    style=style,
                    inflate=(inflate[i] == 1),
                    inflate_style=inflate_style,
                    non_local=(non_local[i] == 1),
                    non_local_cfg=non_local_cfg,
                    norm_cfg=norm_cfg,
                    conv_cfg=conv_cfg,
                    act_cfg=act_cfg,
                    with_cp=with_cp,
                    **kwargs))

        return nn.Sequential(*layers)

    @staticmethod
    def _inflate_conv_params(conv3d, state_dict_2d, module_name_2d,
                             inflated_param_names):
        """Inflate a conv module from 2d to 3d.
        Args:
            conv3d (nn.Module): The destination conv3d module.
            state_dict_2d (OrderedDict): The state dict of pretrained 2d model.
            module_name_2d (str): The name of corresponding conv module in the
                2d model.
            inflated_param_names (list[str]): List of parameters that have been
                inflated.
        """
        weight_2d_name = module_name_2d + '.weight'

        conv2d_weight = state_dict_2d[weight_2d_name]
        kernel_t = conv3d.weight.data.shape[2]

        new_weight = conv2d_weight.data.unsqueeze(2).expand_as(
            conv3d.weight) / kernel_t
        conv3d.weight.data.copy_(new_weight)
        inflated_param_names.append(weight_2d_name)

        if getattr(conv3d, 'bias') is not None:
            bias_2d_name = module_name_2d + '.bias'
            conv3d.bias.data.copy_(state_dict_2d[bias_2d_name])
            inflated_param_names.append(bias_2d_name)

    @staticmethod
    def _inflate_bn_params(bn3d, state_dict_2d, module_name_2d,
                           inflated_param_names):
        """Inflate a norm module from 2d to 3d.
        Args:
            bn3d (nn.Module): The destination bn3d module.
            state_dict_2d (OrderedDict): The state dict of pretrained 2d model.
            module_name_2d (str): The name of corresponding bn module in the
                2d model.
            inflated_param_names (list[str]): List of parameters that have been
                inflated.
        """
        for param_name, param in bn3d.named_parameters():
            param_2d_name = f'{module_name_2d}.{param_name}'
            param_2d = state_dict_2d[param_2d_name]
            if param.data.shape != param_2d.shape:
                warnings.warn(f'The parameter of {module_name_2d} is not'
                              'loaded due to incompatible shapes. ')
                return

            param.data.copy_(param_2d)
            inflated_param_names.append(param_2d_name)

        for param_name, param in bn3d.named_buffers():
            param_2d_name = f'{module_name_2d}.{param_name}'
            # some buffers like num_batches_tracked may not exist in old
            # checkpoints
            if param_2d_name in state_dict_2d:
                param_2d = state_dict_2d[param_2d_name]
                param.data.copy_(param_2d)
                inflated_param_names.append(param_2d_name)

    @staticmethod
    def _inflate_weights(self, logger):
        """Inflate the resnet2d parameters to resnet3d.
        The differences between resnet3d and resnet2d mainly lie in an extra
        axis of conv kernel. To utilize the pretrained parameters in 2d model,
        the weight of conv2d models should be inflated to fit in the shapes of
        the 3d counterpart.
        Args:
            logger (logging.Logger): The logger used to print
                debugging infomation.
        """

        state_dict_r2d = _load_checkpoint(self.pretrained)
        if 'state_dict' in state_dict_r2d:
            state_dict_r2d = state_dict_r2d['state_dict']

        inflated_param_names = []
        for name, module in self.named_modules():
            if isinstance(module, ConvModule):
                # we use a ConvModule to wrap conv+bn+relu layers, thus the
                # name mapping is needed
                if 'downsample' in name:
                    # layer{X}.{Y}.downsample.conv->layer{X}.{Y}.downsample.0
                    original_conv_name = name + '.0'
                    # layer{X}.{Y}.downsample.bn->layer{X}.{Y}.downsample.1
                    original_bn_name = name + '.1'
                else:
                    # layer{X}.{Y}.conv{n}.conv->layer{X}.{Y}.conv{n}
                    original_conv_name = name
                    # layer{X}.{Y}.conv{n}.bn->layer{X}.{Y}.bn{n}
                    original_bn_name = name.replace('conv', 'bn')
                if original_conv_name + '.weight' not in state_dict_r2d:
                    logger.warning(f'Module not exist in the state_dict_r2d'
                                   f': {original_conv_name}')
                else:
                    shape_2d = state_dict_r2d[original_conv_name +
                                              '.weight'].shape
                    shape_3d = module.conv.weight.data.shape
                    if shape_2d != shape_3d[:2] + shape_3d[3:]:
                        logger.warning(f'Weight shape mismatch for '
                                       f': {original_conv_name} : '
                                       f'3d weight shape: {shape_3d}; '
                                       f'2d weight shape: {shape_2d}. ')
                    else:
                        self._inflate_conv_params(module.conv, state_dict_r2d,
                                                  original_conv_name,
                                                  inflated_param_names)

                if original_bn_name + '.weight' not in state_dict_r2d:
                    logger.warning(f'Module not exist in the state_dict_r2d'
                                   f': {original_bn_name}')
                else:
                    self._inflate_bn_params(module.bn, state_dict_r2d,
                                            original_bn_name,
                                            inflated_param_names)

        # check if any parameters in the 2d checkpoint are not loaded
        remaining_names = set(
            state_dict_r2d.keys()) - set(inflated_param_names)
        if remaining_names:
            logger.info(f'These parameters in the 2d checkpoint are not loaded'
                        f': {remaining_names}')

    def inflate_weights(self, logger):
        self._inflate_weights(self, logger)

    def _make_stem_layer(self):
        """Construct the stem layers consists of a conv+norm+act module and a
        pooling layer."""
        self.conv1 = ConvModule(
            self.in_channels,
            self.base_channels,
            kernel_size=self.conv1_kernel,
            stride=(self.conv1_stride_t, self.conv1_stride_s,
                    self.conv1_stride_s),
            padding=tuple([(k - 1) // 2 for k in _triple(self.conv1_kernel)]),
            bias=False,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            act_cfg=self.act_cfg)

        self.maxpool = nn.MaxPool3d(
            kernel_size=(1, 3, 3),
            stride=(self.pool1_stride_t, self.pool1_stride_s,
                    self.pool1_stride_s),
            padding=(0, 1, 1))

        self.pool2 = nn.MaxPool3d(kernel_size=(2, 1, 1), stride=(2, 1, 1))

    def _freeze_stages(self):
        """Prevent all the parameters from being optimized before
        ``self.frozen_stages``."""
        if self.frozen_stages >= 0:
            self.conv1.eval()
            for param in self.conv1.parameters():
                param.requires_grad = False

        for i in range(1, self.frozen_stages + 1):
            m = getattr(self, f'layer{i}')
            m.eval()
            for param in m.parameters():
                param.requires_grad = False

    @staticmethod
    def _init_weights(self, pretrained=None):
        """Initiate the parameters either from existing checkpoint or from
        scratch.
        Args:
            pretrained (str | None): The path of the pretrained weight. Will
                override the original `pretrained` if set. The arg is added to
                be compatible with mmdet. Default: None.
        """
        if pretrained:
            self.pretrained = pretrained
        if isinstance(self.pretrained, str):
            logger = logging.Logger()
            logger.info(f'load model from: {self.pretrained}')

            if self.pretrained2d:
                # Inflate 2D model into 3D model.
                self.inflate_weights(logger)

            else:
                # Directly load 3D model.
                load_checkpoint(
                    self, self.pretrained, strict=False, logger=logger)

        elif self.pretrained is None:
            for m in self.modules():
                if isinstance(m, nn.Conv3d):
                    kaiming_init(m)
                elif isinstance(m, _BatchNorm):
                    constant_init(m, 1)
                    
            if self.zero_init_residual:
                for m in self.modules():
                    if isinstance(m, Bottleneck3d):
                        constant_init(m.conv3.bn, 0)
                    elif isinstance(m, BasicBlock3d):
                        constant_init(m.conv2.bn, 0)
        else:
            raise TypeError('pretrained must be a str or None')

    def init_weights(self, pretrained=None):
        self._init_weights(self, pretrained)

    def forward(self, x):
        """Defines the computation performed at every call.
        Args:
            x (torch.Tensor): The input data.
        Returns:
            torch.Tensor: The feature of the input
            samples extracted by the backbone.
        """
        x = self.conv1(x)
        outs = [x]

        if self.with_pool1:
            x = self.maxpool(x)
        for i, layer_name in enumerate(self.res_layers):
            res_layer = getattr(self, layer_name)
            x = res_layer(x)
            if i == 0 and self.with_pool2:
                x = self.pool2(x)
            if i in self.out_indices:
                outs.append(x)
        if len(outs) == 1:
            return outs[0]

        return tuple(outs)

    def train(self, mode=True):
        """Set the optimization status when training."""
        super().train(mode)
        self._freeze_stages()
        if mode and self.norm_eval:
            for m in self.modules():
                if isinstance(m, _BatchNorm):
                    m.eval()


class ResNet3dLayer(nn.Module):
    """ResNet 3d Layer.
    Args:
        depth (int): Depth of resnet, from {18, 34, 50, 101, 152}.
        pretrained (str | None): Name of pretrained model.
        pretrained2d (bool): Whether to load pretrained 2D model.
            Default: True.
        stage (int): The index of Resnet stage. Default: 3.
        base_channels (int): Channel num of stem output features. Default: 64.
        spatial_stride (int): The 1st res block's spatial stride. Default 2.
        temporal_stride (int): The 1st res block's temporal stride. Default 1.
        dilation (int): The dilation. Default: 1.
        style (str): `pytorch` or `caffe`. If set to "pytorch", the stride-two
            layer is the 3x3 conv layer, otherwise the stride-two layer is
            the first 1x1 conv layer. Default: 'pytorch'.
        all_frozen (bool): Frozen all modules in the layer. Default: False.
        inflate (int): Inflate Dims of each block. Default: 1.
        inflate_style (str): ``3x1x1`` or ``3x3x3``. which determines the
            kernel sizes and padding strides for conv1 and conv2 in each block.
            Default: '3x1x1'.
        conv_cfg (dict): Config for conv layers. required keys are ``type``
            Default: ``dict(type='Conv3d')``.
        norm_cfg (dict): Config for norm layers. required keys are ``type`` and
            ``requires_grad``.
            Default: ``dict(type='BN3d', requires_grad=True)``.
        act_cfg (dict): Config dict for activation layer.
            Default: ``dict(type='ReLU', inplace=True)``.
        norm_eval (bool): Whether to set BN layers to eval mode, namely, freeze
            running stats (mean and var). Default: False.
        with_cp (bool): Use checkpoint or not. Using checkpoint will save some
            memory while slowing down the training speed. Default: False.
        zero_init_residual (bool):
            Whether to use zero initialization for residual block,
            Default: True.
        kwargs (dict, optional): Key arguments for "make_res_layer".
    """

    def __init__(self,
                 depth,
                 pretrained,
                 pretrained2d=True,
                 stage=3,
                 base_channels=64,
                 spatial_stride=2,
                 temporal_stride=1,
                 dilation=1,
                 style='pytorch',
                 all_frozen=False,
                 inflate=1,
                 inflate_style='3x1x1',
                 conv_cfg=dict(type='Conv3d'),
                 norm_cfg=dict(type='BN3d', requires_grad=True),
                 act_cfg=dict(type='ReLU', inplace=True),
                 norm_eval=False,
                 with_cp=False,
                 zero_init_residual=True,
                 **kwargs):

        super().__init__()
        self.arch_settings = ResNet3d.arch_settings
        assert depth in self.arch_settings

        self.make_res_layer = ResNet3d.make_res_layer
        self._inflate_conv_params = ResNet3d._inflate_conv_params
        self._inflate_bn_params = ResNet3d._inflate_bn_params
        self._inflate_weights = ResNet3d._inflate_weights
        self._init_weights = ResNet3d._init_weights

        self.depth = depth
        self.pretrained = pretrained
        self.pretrained2d = pretrained2d
        self.stage = stage
        # stage index is 0 based
        assert 0 <= stage <= 3
        self.base_channels = base_channels

        self.spatial_stride = spatial_stride
        self.temporal_stride = temporal_stride
        self.dilation = dilation

        self.style = style
        self.all_frozen = all_frozen

        self.stage_inflation = inflate
        self.inflate_style = inflate_style
        self.conv_cfg = conv_cfg
        self.norm_cfg = norm_cfg
        self.act_cfg = act_cfg
        self.norm_eval = norm_eval
        self.with_cp = with_cp
        self.zero_init_residual = zero_init_residual

        block, stage_blocks = self.arch_settings[depth]
        stage_block = stage_blocks[stage]
        planes = 64 * 2 ** stage
        inplanes = 64 * 2 ** (stage - 1) * block.expansion

        res_layer = self.make_res_layer(
            block,
            inplanes,
            planes,
            stage_block,
            spatial_stride=spatial_stride,
            temporal_stride=temporal_stride,
            dilation=dilation,
            style=self.style,
            norm_cfg=self.norm_cfg,
            conv_cfg=self.conv_cfg,
            act_cfg=self.act_cfg,
            inflate=self.stage_inflation,
            inflate_style=self.inflate_style,
            with_cp=with_cp,
            **kwargs)

        self.layer_name = f'layer{stage + 1}'
        self.add_module(self.layer_name, res_layer)

    def inflate_weights(self, logger):
        self._inflate_weights(self, logger)

    def _freeze_stages(self):
        """Prevent all the parameters from being optimized before
        ``self.frozen_stages``."""
        if self.all_frozen:
            layer = getattr(self, self.layer_name)
            layer.eval()
            for param in layer.parameters():
                param.requires_grad = False

    def init_weights(self, pretrained=None):
        self._init_weights(self, pretrained)

    def forward(self, x):
        """Defines the computation performed at every call.
        Args:
            x (torch.Tensor): The input data.
        Returns:
            torch.Tensor: The feature of the input
            samples extracted by the backbone.
        """
        res_layer = getattr(self, self.layer_name)
        out = res_layer(x)
        return out

    def train(self, mode=True):
        """Set the optimization status when training."""
        super().train(mode)
        self._freeze_stages()
        if mode and self.norm_eval:
            for m in self.modules():
                if isinstance(m, _BatchNorm):
                    m.eval()


class CSNBottleneck3d(Bottleneck3d):
    """Channel-Separated Bottleneck Block.
    This module is proposed in
    "Video Classification with Channel-Separated Convolutional Networks"
    Link: https://arxiv.org/pdf/1711.11248.pdf
    Args:
        inplanes (int): Number of channels for the input in first conv3d layer.
        planes (int): Number of channels produced by some norm/conv3d layers.
        bottleneck_mode (str): Determine which ways to factorize a 3D
            bottleneck block using channel-separated convolutional networks.
                If set to 'ip', it will replace the 3x3x3 conv2 layer with a
                1x1x1 traditional convolution and a 3x3x3 depthwise
                convolution, i.e., Interaction-preserved channel-separated
                bottleneck block.
                If set to 'ir', it will replace the 3x3x3 conv2 layer with a
                3x3x3 depthwise convolution, which is derived from preserved
                bottleneck block by removing the extra 1x1x1 convolution,
                i.e., Interaction-reduced channel-separated bottleneck block.
            Default: 'ir'.
        args (position arguments): Position arguments for Bottleneck.
        kwargs (dict, optional): Keyword arguments for Bottleneck.
    """

    def __init__(self,
                 inplanes,
                 planes,
                 *args,
                 bottleneck_mode='ir',
                 **kwargs):
        super(CSNBottleneck3d, self).__init__(inplanes, planes, *args,
                                              **kwargs)
        self.bottleneck_mode = bottleneck_mode
        conv2 = []
        if self.bottleneck_mode == 'ip':
            conv2.append(
                ConvModule(
                    planes,
                    planes,
                    1,
                    stride=1,
                    bias=False,
                    conv_cfg=self.conv_cfg,
                    norm_cfg=self.norm_cfg,
                    act_cfg=None))
        conv2_kernel_size = self.conv2.conv.kernel_size
        conv2_stride = self.conv2.conv.stride
        conv2_padding = self.conv2.conv.padding
        conv2_dilation = self.conv2.conv.dilation
        conv2_bias = bool(self.conv2.conv.bias)
        self.conv2 = ConvModule(
            planes,
            planes,
            conv2_kernel_size,
            stride=conv2_stride,
            padding=conv2_padding,
            dilation=conv2_dilation,
            bias=conv2_bias,
            conv_cfg=self.conv_cfg,
            norm_cfg=self.norm_cfg,
            act_cfg=self.act_cfg,
            groups=planes)
        conv2.append(self.conv2)
        self.conv2 = nn.Sequential(*conv2)


class ResNet3dCSN(ResNet3d):
    """ResNet backbone for CSN.
    Args:
        depth (int): Depth of ResNetCSN, from {18, 34, 50, 101, 152}.
        pretrained (str | None): Name of pretrained model.
        temporal_strides (tuple[int]):
            Temporal strides of residual blocks of each stage.
            Default: (1, 2, 2, 2).
        conv1_kernel (tuple[int]): Kernel size of the first conv layer.
            Default: (3, 7, 7).
        conv1_stride_t (int): Temporal stride of the first conv layer.
            Default: 1.
        pool1_stride_t (int): Temporal stride of the first pooling layer.
            Default: 1.
        norm_cfg (dict): Config for norm layers. required keys are `type` and
            `requires_grad`.
            Default: dict(type='BN3d', requires_grad=True, eps=1e-3).
        inflate_style (str): `3x1x1` or `3x3x3`. which determines the kernel
            sizes and padding strides for conv1 and conv2 in each block.
            Default: '3x3x3'.
        bottleneck_mode (str): Determine which ways to factorize a 3D
            bottleneck block using channel-separated convolutional networks.
                If set to 'ip', it will replace the 3x3x3 conv2 layer with a
                1x1x1 traditional convolution and a 3x3x3 depthwise
                convolution, i.e., Interaction-preserved channel-separated
                bottleneck block.
                If set to 'ir', it will replace the 3x3x3 conv2 layer with a
                3x3x3 depthwise convolution, which is derived from preserved
                bottleneck block by removing the extra 1x1x1 convolution,
                i.e., Interaction-reduced channel-separated bottleneck block.
            Default: 'ip'.
        kwargs (dict, optional): Key arguments for "make_res_layer".
    """

    def __init__(self,
                 depth,
                 pretrained,
                 temporal_strides=(1, 2, 2, 2),
                 conv1_kernel=(3, 7, 7),
                 conv1_stride_t=1,
                 pool1_stride_t=1,
                 norm_cfg=dict(type='BN3d', requires_grad=True, eps=1e-3),
                 inflate_style='3x3x3',
                 bottleneck_mode='ir',
                 bn_frozen=False,
                 **kwargs):
        self.arch_settings = {
            # 18: (BasicBlock3d, (2, 2, 2, 2)),
            # 34: (BasicBlock3d, (3, 4, 6, 3)),
            50: (CSNBottleneck3d, (3, 4, 6, 3)),
            101: (CSNBottleneck3d, (3, 4, 23, 3)),
            152: (CSNBottleneck3d, (3, 8, 36, 3))
        }
        self.bn_frozen = bn_frozen
        if bottleneck_mode not in ['ip', 'ir']:
            raise ValueError(f'Bottleneck mode must be "ip" or "ir",'
                             f'but got {bottleneck_mode}.')
        super(ResNet3dCSN, self).__init__(
            depth,
            pretrained,
            temporal_strides=temporal_strides,
            conv1_kernel=conv1_kernel,
            conv1_stride_t=conv1_stride_t,
            pool1_stride_t=pool1_stride_t,
            norm_cfg=norm_cfg,
            inflate_style=inflate_style,
            bottleneck_mode=bottleneck_mode,
            **kwargs)

    def train(self, mode=True):
        super(ResNet3d, self).train(mode)
        self._freeze_stages()
        if mode and self.norm_eval:
            for m in self.modules():
                if isinstance(m, _BatchNorm):
                    m.eval()
                    if self.bn_frozen:
                        for param in m.parameters():
                            param.requires_grad = False


In [ ]:
def filterMask(mask, thresh):
    for i in range(mask.shape[0]):
        if np.sum(mask[i]) < thresh:
            mask[i] *= 0
    return mask

def findSingleBoundingBoxPerClass(prediction : np.array, padding : float) -> dict:
    bounding_boxes = {}
    classes = np.unique(prediction)
    z, y, x = prediction.shape
    for c in classes:
        if c == 0 or c == 5:  # фон и кишечник
            continue
        mask = (prediction == c).astype(bool)
        if c == 1:  # liver
            mask = (prediction == c).astype(np.uint8)
            mask = filterMask(mask, thresh=500)
        coords = np.argwhere(mask)

        z_min, y_min, x_min = coords.min(axis=0)
        z_max, y_max, x_max = coords.max(axis=0)

        x_padding = int((x_max - x_min) * padding)
        y_padding = int((y_max - y_min) * padding)
        z_padding = int((z_max - z_min) * padding)

        x_min = max(x_min - x_padding, 0) / x
        y_min = max(y_min - y_padding, 0) / y
        z_min = max(z_min - z_padding, 0) / z
        
        x_max = min(x_max + x_padding, prediction.shape[2]) / x
        y_max = min(y_max + y_padding, prediction.shape[1]) / y
        z_max = min(z_max + z_padding, prediction.shape[0]) / z
        
        bounding_boxes[c] = x_min, x_max, y_min, y_max, z_min, z_max
    
    return bounding_boxes

def getCoords(full_cube : np.array, bounding_boxes : dict, label : int, padding : float) -> Tuple[int]:
    x_start, x_end, y_start, y_end, z_start, z_end = bounding_boxes[label]
    
    x_start, x_end = x_start * full_cube.shape[2], x_end * full_cube.shape[2]
    x_start, x_end = max(x_start - ((x_end - x_start) * padding), 0) , min(x_end + ((x_end - x_start) * padding), full_cube.shape[2])
    
    y_start, y_end = y_start * full_cube.shape[1], y_end * full_cube.shape[1]
    y_start, y_end = max(y_start - ((y_end - y_start) * padding), 0) , min(y_end + ((y_end - y_start) * padding), full_cube.shape[1])
    
    z_start, z_end = z_start * full_cube.shape[0], z_end * full_cube.shape[0]
    z_start, z_end = max(z_start - ((z_end - z_start) * padding), 0) , min(z_end + ((z_end - z_start) * padding), full_cube.shape[0])
    
    return int(x_start), int(x_end), int(y_start), int(y_end), int(z_start), int(z_end)

def cropOrgan(full_cube : np.array, coords : Tuple[int]) -> np.array:
    x_start, x_end, y_start, y_end, z_start, z_end = coords
#     print(coords)
    
    organ = full_cube[z_start : z_end, y_start : y_end, x_start : x_end]
    
    image_mean = organ.mean()
    image_std = organ.std()
    organ = (organ - image_mean) / image_std
    
    return organ

def getOrganCube(full_cube : np.array, bounding_boxes : dict, label : int, padding : float) -> np.array:
    coords = getCoords(full_cube, bounding_boxes, label, padding)
    return cropOrgan(full_cube, coords)

def resizeSitk(image, output_size, is_mask=False):
    original_size = image.GetSize()
    original_spacing = image.GetSpacing()

    new_spacing = [
        original_spacing[i] * (original_size[i] / output_size[i]) for i in range(3)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(output_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(
        sitk.sitkNearestNeighbor if is_mask else sitk.sitkLinear
    )

    resized_image = resampler.Execute(image)

    return resized_image

def getImageFromArray(image : np.array) -> sitk.Image:
    image  = sitk.GetImageFromArray(image)
    image.SetOrigin((0, 0, 0))
    spacing = image.GetSpacing()
    image.SetSpacing(spacing)
    return image
    
def concatKidneys(left : np.array, right : np.array) -> np.array:
    z_l, y_l, x_l = left.shape
    z_r, y_r, x_r = right.shape
    z_max = max(z_l, z_r)
    y_max = max(y_l, y_r)
    
    left_sitk = getImageFromArray(left)
    right_sitk = getImageFromArray(right)
    
    left_resized = resizeSitk(left_sitk, (x_l, y_max, z_max))
    right_resized = resizeSitk(right_sitk, (x_r, y_max, z_max))
    
    left = sitk.GetArrayFromImage(left_resized)
    right = sitk.GetArrayFromImage(right_resized)
    
    kidneys = np.concatenate((left, right), axis = 2)
    image_mean = kidneys.mean()
    image_std = kidneys.std()
    kidneys = (kidneys - image_mean) / image_std
    return kidneys

def getBestScanId(path_to_scan : List[str]) -> str:
    if len(path_to_scan) == 1:
        return path_to_scan[0]
    if len(os.listdir(path_to_scan[0])) > 200:
        return path_to_scan[0] 
    if len(os.listdir(path_to_scan[1])) > len(os.listdir(path_to_scan[0])):
        return path_to_scan[1]
    return path_to_scan[0]

In [ ]:
class DatasetYoloKaggle(Dataset):
    def __init__(
        self,
        cube : np.array,
    ):
        self.cube = cube
       
    def __getitem__(self, i):
        image = self.cube[i]
        if image.shape[0] != 512 or image.shape[1] != 512:
            image = cv2.resize(image, (512, 512), interpolation = cv2.INTER_AREA)
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB).T
        return image
        
    def __len__(self):
        return self.cube.shape[0]
    
class ExtravasationPrediction():
    def __init__(
        self,
        path_to_yolo_weights : str,
        path_to_pred_models : str,
        device : str
    ):
        self.YOLO = YOLO(path_to_yolo_weights).to(device)
        self.prediction_models = self.initializeExtravasationModels(path_to_pred_models)

    def initializeExtravasationModels(self, path_to_models : str):
        lineag_regressions = []
        shifted_lineag_regressions = []
        random_forests = []
        shifted_random_forests = []
        for model_path in glob.glob(os.path.join(path_to_models, "*.pkl")):
            if model_path.split("/")[-1].split("_")[0] == "LR":
                if model_path.split("/")[-1].split("_")[1] != "shifted":
                    LR = joblib.load(model_path)
                    lineag_regressions.append(LR)
                else:
                    LR = joblib.load(model_path)
                    shifted_lineag_regressions.append(LR)
                
        full_train_data = pd.read_csv("/kaggle/input/extr-models/data/data.csv")
        shifted_train_data = pd.read_csv("/kaggle/input/extr-models/data/shifted_labels.csv")
        feature_cols = ['scores_mean', 'scores_max', 'scores_median', 'scores_std', 'scores_std_not_null',
                        'shape', "feature_1", 'squares_max', 'squares_median', 'squares_std_not_null']
        BRF = BalancedRandomForestClassifier(n_estimators=100,
                                     criterion="gini",
                                     max_depth=None,
                                     min_samples_split=2,
                                     min_samples_leaf=1,
                                     min_weight_fraction_leaf=0.,
                                     max_features='sqrt',
                                     max_leaf_nodes=None,
                                     min_impurity_decrease=0.,
                                     bootstrap=True,
                                     oob_score=False,
                                     sampling_strategy="auto",
                                     replacement=False,
                                     random_state=21,
                                     verbose=0,
                                     warm_start=False,
                                     class_weight={0 : 1, 1 : 6},
                                     ccp_alpha=0.,
                                     max_samples=None
                                    )
        N_SPLITS = len(full_train_data.fold.unique())
        for i in range(N_SPLITS):
            X_train = full_train_data[full_train_data.fold != i][feature_cols]
            y_train = full_train_data[full_train_data.fold != i]["label"]
            X_val = full_train_data[full_train_data.fold == i][feature_cols]
            y_val = full_train_data[full_train_data.fold == i]["label"]
            fit_BRF = BRF.fit(X_train, y_train)
            random_forests.append(fit_BRF)
        fit_BRF = BRF.fit(full_train_data[feature_cols], full_train_data["label"])
        random_forests.append(fit_BRF)
            
        N_SPLITS = len(shifted_train_data.fold.unique())
        for i in range(N_SPLITS):
            X_train = shifted_train_data[shifted_train_data.fold != i][feature_cols]
            y_train = shifted_train_data[shifted_train_data.fold != i]["label"]
            X_val = shifted_train_data[shifted_train_data.fold == i][feature_cols]
            y_val = shifted_train_data[shifted_train_data.fold == i]["label"]
            fit_BRF = BRF.fit(X_train, y_train)
            shifted_random_forests.append(fit_BRF)
        fit_BRF = BRF.fit(shifted_train_data[feature_cols], shifted_train_data["label"])
        shifted_random_forests.append(fit_BRF)

        return (lineag_regressions, random_forests, shifted_lineag_regressions, shifted_random_forests)
    
    def TTA(self, model, batch : np.array, imgsz : int, conf : float):
        shape = batch.shape[0]
        confs, squares = np.zeros(shape), np.zeros(shape)
        for i in range(4):
            batch_rot = torch.rot90(batch, k=i, dims=[2, 3])

            preds = model.predict(batch_rot, imgsz=imgsz, conf=conf, verbose=False)
            for index, pred in enumerate(preds):
                if len(pred.boxes.conf) != 0:
                    """
                    беру только 1й предикт по конфиденсу и площади
                    """
                    pred_confs = pred.boxes.conf.detach().cpu().numpy()[0]
                    confs[index] += pred_confs / 4
                    pred_xywhn = pred.boxes.xywhn.detach().cpu().numpy()[:, 2:]
                    squares[index] += (pred_xywhn[:, 0][0] * pred_xywhn[:, 1][0]) / 4

        return confs, squares
    
    def getYoloScores(self, model, batch : np.array, imgsz : int = 512, conf : float = 0.01, is_tta : bool = False):
        if is_tta:
            confs, squares = self.TTA(model, batch, imgsz, conf)
            return confs, squares
        else:
            shape = batch.shape[0]
            confs, squares = np.zeros(shape), np.zeros(shape)
            preds = model.predict(batch, imgsz=imgsz, conf=conf, verbose=False)
            for index, pred in enumerate(preds):
                if len(pred.boxes.conf) != 0:
                    """
                    беру только 1й предикт по конфиденсу и площади
                    """
                    pred_confs = pred.boxes.conf.detach().cpu().numpy()[0]
                    confs[index] += pred_confs
                    pred_xywhn = pred.boxes.xywhn.detach().cpu().numpy()[:, 2:]
                    squares[index] += pred_xywhn[:, 0][0] * pred_xywhn[:, 1][0]
            return confs, squares

    def getYoloPredictions(self, cube : np.array, batch_size : int, is_tta : bool = False):
        model = self.YOLO
        dataset = DatasetYoloKaggle(cube)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
        scores = []
        squares = []
        for batch in dataloader:
            score, square = self.getYoloScores(model, batch, conf=0.1, is_tta=is_tta)
            scores.extend(score)
            squares.extend(square)
        return scores, squares
    
    def f(self, x, th = 0.35):
        counter = 0
        m = 0
        for i in x:
            if i > th:
                counter += 1
            else:
                counter = 0
            if counter > m:
                m = counter
        return m / len(x)

    def getFeatures(self, scores : np.array, squares : np.array):
        scores = np.array(scores)
        squares = np.array(squares)
        scores_not_null = scores[scores!=0.0]
        scores_mean = scores_not_null.mean()
        scores_max = scores.max()
        scores_median = np.median(scores_not_null)
        scores_std = scores.std()
        scores_std_not_null = scores_not_null.std()
        shape = len(scores) / 500
        my_feature = (self.f(scores, 0.35) * scores_max / (scores_not_null.min() + 1e-6)) * (scores_std_not_null/(scores_std + 1e-6))
        squares_max = squares.max()
        squares_median = np.median(squares[squares!=0.0])
        squares_std_not_null = squares[squares!=0.0].std()
        return pd.DataFrame({ 'scores_mean' : scores_mean, 'scores_max' : scores_max, 'scores_median' : scores_median,
                 'scores_std' : scores_std, 'scores_std_not_null' : scores_std_not_null, 'shape' : shape,
                 'feature_1' : my_feature, 'squares_max' : squares_max, 'squares_median' : squares_median, 'squares_std_not_null' : squares_std_not_null}, index=[0])
        

    def getExtravasationPrediction(self, cube : np.array, batch_size : int, use_shifted_data : bool = False):
        scores, squares = self.getYoloPredictions(cube, batch_size, is_tta=False)
        features = self.getFeatures(scores, squares)
        
        NUM_MODELS = 5
        prediction_models = self.prediction_models
        linear_regressions = prediction_models[0]
        random_forests = prediction_models[1]
        shifted_linear_regressions = prediction_models[2]
        shifted_random_forests = prediction_models[3]
        LR_scores = np.array([0., 0.])
        RF_scores = np.array([0., 0.])
        for fold_num in range(NUM_MODELS):
            LR = linear_regressions[fold_num]
            RF = random_forests[fold_num]
            LR_pred = LR.predict_proba(features)[0]
            RF_pred = RF.predict_proba(features)[0]
            LR_scores += LR_pred / NUM_MODELS
            RF_scores += RF_pred / NUM_MODELS
        score = 0.5 * LR_scores + 0.5 * RF_scores
        if use_shifted_data:
            LR_scores_shifted = np.array([0., 0.])
            RF_scores_shifted = np.array([0., 0.])
            for fold_num in range(NUM_MODELS):
                LR = shifted_linear_regressions[fold_num]
                RF = shifted_random_forests[fold_num]
                LR_pred = LR.predict_proba(features)[0]
                RF_pred = RF.predict_proba(features)[0]
                LR_scores_shifted += LR_pred / NUM_MODELS
                RF_scores_shifted += RF_pred / NUM_MODELS
            score_shifted = 0.5 * LR_scores_shifted + 0.5 * RF_scores_shifted
            score = 0.8 * score + 0.2 * score_shifted
        return score

In [ ]:
class Config:
    #dataset
    ROOT = "data"
    path_to_images   : str = f"{ROOT}/train_images"
#     path_to_meta_csv : str = f"{ROOT}/seg_meta.csv"
    path_to_meta_csv : str = f"{ROOT}/train_series_meta.csv" # for all images
        
    path_to_masks    : str = f"{ROOT}/segmentations_png"
    fold       : int = 0
    height     : int = 256
    width      : int = 256
    slice_size : int = 112
    num_workes : int = 6
    
    #training
    num_classes      : int = 6
    batch_size       : int = 1
    wandb_project    : str = 'RSNA_segmentation'
    default_root_dir : str = 'seg'
    checkpoints_dir  : str = 'seg/checkpoints'
    lr               : float = 1e-3
    max_epochs       : int = 120

In [ ]:
import torch
from torch import nn
from torch.nn import Conv3d

# from zoo.resnet3d_csn import ResNet3dCSN

encoder_params = {
    "r152ir": {
        "filters": [64, 256, 512, 1024, 2048],
        "decoder_filters": [40, 64, 128, 256],
    },
    "r50ir": {
        "filters": [64, 256, 512, 1024, 2048],
        "decoder_filters": [40, 64, 128, 256],
    },
}


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv3d(in_channels, out_channels, 3, padding=1),
            nn.SiLU(inplace=True),
        )

    def forward(self, x):
        return self.layer(x)


class LastDecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Upsample(scale_factor=(1, 2, 2)),
            nn.Conv3d(in_channels, out_channels, 3, padding=1),
            nn.SiLU(inplace=True),
        )

    def forward(self, x):
        return self.layer(x)


class ConcatBottleneck(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, 3, padding=1), nn.SiLU(inplace=True)
        )

    def forward(self, dec, enc):
        x = torch.cat([dec, enc], dim=1)
        return self.seq(x)


class Decoder(nn.Module):
    def __init__(
        self,
        decoder_filters,
        filters,
        upsample_filters=32,
        decoder_block=DecoderBlock,
        bottleneck=ConcatBottleneck,
    ):
        super().__init__()
        self.decoder_filters = decoder_filters
        self.filters = filters
        self.decoder_block = decoder_block
        self.decoder_stages = nn.ModuleList(
            [self._get_decoder(idx) for idx in range(0, len(decoder_filters))]
        )
        self.bottlenecks = nn.ModuleList(
            [
                bottleneck(self.filters[-i - 2] + f, f)
                for i, f in enumerate(reversed(decoder_filters))
            ]
        )
        self.last_block = None
        if upsample_filters:
            self.last_block = LastDecoderBlock(
                decoder_filters[0], out_channels=upsample_filters
            )
        else:
            self.last_block = nn.Upsample(scale_factor=(1, 2, 2), mode="trilinear")

    def forward(self, encoder_results: list):
        x = encoder_results[0]
        bottlenecks = self.bottlenecks
        for idx, bottleneck in enumerate(bottlenecks):
            rev_idx = -(idx + 1)
            x = self.decoder_stages[rev_idx](x)
            x = bottleneck(x, encoder_results[-rev_idx])
        if self.last_block:
            x = self.last_block(x)
        return x

    def _get_decoder(self, layer):
        idx = layer + 1
        if idx == len(self.decoder_filters):
            in_channels = self.filters[idx]
        else:
            in_channels = self.decoder_filters[idx]
        return self.decoder_block(in_channels, self.decoder_filters[max(layer, 0)])


def _initialize_weights(module):
    for m in module.modules():
        if (
            isinstance(m, nn.Conv2d)
            or isinstance(m, nn.Conv3d)
            or isinstance(m, nn.ConvTranspose2d)
            or isinstance(m, nn.Linear)
        ):
            m.weight.data = nn.init.kaiming_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm3d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()


class Conv2P1DBnAct(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int, act=nn.ReLU, bn=False
    ) -> None:
        super().__init__()
        self.conv1_s = nn.Conv3d(
            in_channels,
            out_channels,
            kernel_size=(1, 3, 3),
            stride=(1, 1, 1),
            padding=(0, 1, 1),
            bias=not bn,
        )
        self.bn1_s = nn.BatchNorm3d(out_channels) if bn else nn.Identity()
        self.conv1_t = nn.Conv3d(
            out_channels,
            out_channels,
            kernel_size=(3, 1, 1),
            stride=(1, 1, 1),
            padding=(1, 0, 0),
            bias=not bn,
        )
        self.bn1_t = nn.BatchNorm3d(out_channels) if bn else nn.Identity()
        self.relu = act(inplace=True)

    def forward(self, x):
        x = self.conv1_s(x)
        x = self.bn1_s(x)
        x = self.relu(x)
        x = self.conv1_t(x)
        x = self.bn1_t(x)
        x = self.relu(x)
        return x


class DecoderBlockConv2P1D(nn.Module):
    def __init__(self, in_channels, out_channels, scale_factor=(2, 2, 2)):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Upsample(scale_factor=scale_factor),
            Conv2P1DBnAct(in_channels, out_channels, bn=False),
        )

    def forward(self, x):
        return self.layer(x)


class ConcatBottleneckConv2P1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.seq = nn.Sequential(Conv2P1DBnAct(in_channels, out_channels, bn=False))

    def forward(self, dec, enc):
        x = torch.cat([dec, enc], dim=1)
        return self.seq(x)


class CSNDecoder(Decoder):
    def _get_decoder(self, layer):
        idx = layer + 1
        if idx == len(self.decoder_filters):
            in_channels = self.filters[idx]
        else:
            in_channels = self.decoder_filters[idx]
        if idx == 1:
            return self.decoder_block(
                in_channels, self.decoder_filters[max(layer, 0)], (1, 2, 2)
            )
        else:
            return self.decoder_block(in_channels, self.decoder_filters[max(layer, 0)])


class ResNet3dCSN2P1D(nn.Module):
    def __init__(self, encoder="r152ir", num_classes=1) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.decoder = CSNDecoder(
            decoder_filters=encoder_params[encoder]["decoder_filters"],
            filters=encoder_params[encoder]["filters"],
            decoder_block=DecoderBlockConv2P1D,
            bottleneck=ConcatBottleneckConv2P1D,
        )
        self.final = Conv3d(32, out_channels=num_classes, kernel_size=1)
        _initialize_weights(self)

        self.backbone = ResNet3dCSN(
            pretrained2d=False,
            pretrained=None,
            depth=int(encoder[1:-2]),
            with_pool2=False,
            bottleneck_mode="ir",
            norm_eval=False,
            zero_init_residual=False,
        )

    def forward(self, x):
        x = x.repeat(1, 3, 1, 1, 1)[:, :, :, :, :]
        encoder_results = list(reversed(self.backbone(x)))
        x = self.decoder(encoder_results)
        x = self.final(x)
        return {"mask": x}


In [ ]:
def get_files(folder, ext=''):
    paths = []
    for item in os.listdir(folder):
        item_path = os.path.join(folder, item)
        if os.path.isfile(item_path) and item.endswith(ext):
            paths.append(item_path)
        elif os.path.isdir(item_path):
            paths.extend(get_files(item_path, ext))
    return paths

def standardize_pixel_array(dcm: pydicom.dataset.FileDataset) -> np.ndarray:
    # Correct DICOM pixel_array if PixelRepresentation == 1.
    pixel_array = dcm.pixel_array
    if dcm.PixelRepresentation == 1:
        bit_shift = dcm.BitsAllocated - dcm.BitsStored
        dtype = pixel_array.dtype 
        pixel_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
    return pixel_array

def read_dicom(path: str, fix_monochrome: bool = True) -> np.ndarray:
    dicom = pydicom.dcmread(path)
    data = standardize_pixel_array(dicom)
    
    # find rescale params
    if ("RescaleIntercept" in dicom) and ("RescaleSlope" in dicom):
        intercept = float(dicom.RescaleIntercept)
        slope = float(dicom.RescaleSlope)

    # find clipping params
    center = int(dicom.WindowCenter)
    width = int(dicom.WindowWidth)
    
    low = center - width / 2
    high = center + width / 2

    data = (data * slope) + intercept
    
    data = np.clip(data, low, high)

    data = data - np.min(data)
    data = data / (np.max(data) + 1e-5)
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = 1.0 - data
    return (data * 255).astype(np.uint8)

def get_cube(path : str, koeff : int) -> np.ndarray:
    dicom_paths = get_files(path, ext='.dcm')
    sorted_dicom_paths = sorted(dicom_paths, key=lambda s: int(s.split("/")[-1].split(".")[0]))
    cube = []
    for i, image_path in enumerate(sorted_dicom_paths):
        if i % koeff != 0:
            continue
        image = read_dicom(image_path)
        cube.append(image)
    return np.stack(cube)

def get_dicom_tags(path : str) -> pd.DataFrame:
    dicoms_tags = pd.read_parquet(path)
    dicoms_tags["z"]   = dicoms_tags.ImagePositionPatient.apply(lambda x: float(x.split(", ")[-1][:-1]))
    dicoms_tags["patient_id"] = dicoms_tags.path.apply(lambda x: x.split("/")[1])
    dicoms_tags["series_id"] = dicoms_tags.path.apply(lambda x: x.split("/")[2])
    dicoms_tags["slice_id"] = dicoms_tags.path.apply(lambda x: int(x.split("/")[-1].split(".")[0]))
    return dicoms_tags

def get_spacing(df : pd.DataFrame, patient_id : str, series_id : str) -> Tuple[float]:
    try:
        rows = df[(df.patient_id == patient_id) & (df.series_id == series_id)][["SliceThickness", "z", "slice_id"]][:2]
        thikness = rows.SliceThickness[:1].item()

        diff = rows.diff()[-1:]

        spacing = round((diff.z / ((diff.slice_id) + 1e-6)).item(), 3)
        return abs(spacing), thikness
    except:
        return 2.5, 2.5


def resize_sitk(image, output_size : Tuple[int], is_mask : bool = False):
    original_size = image.GetSize()
    original_spacing = image.GetSpacing()
    
    new_spacing = [
        original_spacing[i] * (original_size[i] / output_size[i]) for i in range(3)
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetSize(output_size)
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetTransform(sitk.Transform())
    resampler.SetInterpolator(
        sitk.sitkNearestNeighbor if is_mask else sitk.sitkLinear
    )

    resized_image = resampler.Execute(image)

    return resized_image

def normalize_image(image : np.array) -> np.array:
    image_mean = image.mean()
    image_std = image.std()
    image = (image - image_mean) / image_std
    return image

def resize_cube_sitk(image : np.array, width : int, height : int, slice_size : int) -> np.array:
    image_sitk = sitk.GetImageFromArray(image)
    resized_image_sitk = resize_sitk(
            image_sitk,
            (width, height, slice_size),
            is_mask=False,
            )
    image = sitk.GetArrayFromImage(resized_image_sitk)
    return image

def prepare_cube_for_segmentation(image : np.array, width : int, height : int, slice_size : int) -> np.array:
    resized_image = resize_cube_sitk(image, width, height, slice_size)
    normalized_image = normalize_image(resized_image)
    image = np.expand_dims(normalized_image, 0)
    sample = {}
    sample["image"] = torch.from_numpy(image.astype(np.float32))
    return sample

def padding3D(image : sitk.Image, width, height, slice_size, full_zeroes=False) -> np.ndarray:
    shape = (width, height, slice_size)
    if any([image.GetSize()[i] > shape[i] for i in range(3)]):
        resize_factor = min([shape[i] / image.GetSize()[i] for i in range(3)])
        new_size = [int(image.GetSize()[i] * resize_factor) for i in range(3)]
        image = resize_sitk(image, new_size)
        
    def pad_helper(img_array: np.ndarray, shape: tuple, is_mask=False) -> np.ndarray:
        # Determine the value to pad with
        initial_value = np.min(img_array) if full_zeroes or is_mask else 0

        # Create an empty array with the desired shape
        pad_img_array = np.full(shape, initial_value, dtype=img_array.dtype)

        # Calculate the starting indices for the input image (to center it)
        offset = [(shape[i] - img_array.shape[i]) // 2 for i in range(3)]

        # Insert the input image into the empty array
        pad_img_array[
            offset[0] : offset[0] + img_array.shape[0],
            offset[1] : offset[1] + img_array.shape[1],
            offset[2] : offset[2] + img_array.shape[2],
        ] = img_array
        
        return pad_img_array
    
    padded_image = pad_helper(sitk.GetArrayFromImage(image).T, shape)

    return padded_image
        

def getOrganForClassification(image, width, height, slice_size):
    image_sitk = sitk.GetImageFromArray(image)
    image = padding3D(image_sitk, width, height, slice_size)
    image = np.stack([image, image, image])
    image = np.expand_dims(image, 0)
    sample = {}
    sample["image"] = torch.from_numpy(image.astype(np.float32))
    return sample

In [ ]:
TYPE            = "test"
IMAGES_PATH     = f"/kaggle/input/rsna-2023-abdominal-trauma-detection/{TYPE}_images"
DICOM_TAGS_PATH = f"/kaggle/input/rsna-2023-abdominal-trauma-detection/{TYPE}_dicom_tags.parquet"
DEVICE          = "cuda"

In [ ]:
encoder_name = "r50ir"
modelSegm = ResNet3dCSN2P1D(encoder=encoder_name, num_classes=Config.num_classes).to(torch.float32).to(DEVICE)
modelSegm.load_state_dict(torch.load("/kaggle/input/rsna-models/seg_fold0.pt"))

In [ ]:
import torch
import torch.nn as nn
from torchvision.models.video.resnet import VideoResNet


def _initialize_weights(module):
    for m in module.modules():
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            m.weight.data = nn.init.kaiming_normal_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
            
class ClassifierResNet3dCSN2P1D(nn.Module):
    def __init__(
        self,
        encoder="r50ir",
        pool="avg",
        norm_eval=False,
        num_classes=1,
        head_dropout=0.2,
    ) -> None:
        super().__init__()

        self.head = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.GELU(),
            nn.Dropout(head_dropout),
            nn.Linear(1024, num_classes),
        )
        self.avg_pool = (
            nn.AdaptiveAvgPool3d((1, 1, 1))
            if pool == "avg"
            else nn.AdaptiveMaxPool3d((1, 1, 1))
        )
        self.dropout = nn.Dropout(0.5)
        _initialize_weights(self)

        self.backbone = ResNet3dCSN(
            pretrained2d=False,
            pretrained=None,
            depth=int(encoder[1:-2]),
            with_pool2=False,
            bottleneck_mode=encoder[-2:],
            norm_eval=norm_eval,
            zero_init_residual=False,
        )

    def forward(self, x):
        if x.size(1) == 1:
            x = x.repeat(1, 3, 1, 1, 1)[:, :, :, :, :]
#             print("changed")
        x = self.backbone(x)[-1]
        x = self.avg_pool(x)
        x = self.dropout(x)
        x = x.flatten(1)
        x = self.head(x)
        return x


In [ ]:
kidneyModel_1  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
kidneyModel_1.load_state_dict(torch.load("/kaggle/input/spleen-weights/kidneys/kidneys_fold1.pt"))
kidneyModel_1 = kidneyModel_1.eval()

kidneyModel_2  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
kidneyModel_2.load_state_dict(torch.load("/kaggle/input/spleen-weights/kidneys/kidneys_fold2.pt"))
kidneyModel_2 = kidneyModel_2.eval()

kidneyModel_4  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
kidneyModel_4.load_state_dict(torch.load("/kaggle/input/spleen-weights/kidneys/kidneys_fold4.pt"))
kidneyModel_4 = kidneyModel_4.eval()
#kidney (192, 112, 64)

In [ ]:
spleenModel_0  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
spleenModel_0.load_state_dict(torch.load("/kaggle/input/spleen-weights/spleen_fold0.pt"))
spleenModel_0 = spleenModel_0.eval()

spleenModel_1  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
spleenModel_1.load_state_dict(torch.load("/kaggle/input/spleen-weights/spleen_fold1.pt"))
spleenModel_1 = spleenModel_1.eval()

spleenModel_2  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
spleenModel_2.load_state_dict(torch.load("/kaggle/input/spleen-weights/spleen_fold2.pt"))
spleenModel_2 = spleenModel_2.eval()

spleenModel_3  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
spleenModel_3.load_state_dict(torch.load("/kaggle/input/spleen-weights/spleen_fold3.pt"))
spleenModel_3 = spleenModel_3.eval()

spleenModel_4  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
spleenModel_4.load_state_dict(torch.load("/kaggle/input/spleen-weights/spleen_fold4.pt"))
spleenModel_4 = spleenModel_4.eval()

# with open("/kaggle/input/spleen-weights/spleen.json", "r") as f:
#     spleenMeta = json.load(f)
    
#spleen (z, y, x) (64, 128, 128)

In [ ]:
liverModel_0  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
liverModel_0.load_state_dict(torch.load("/kaggle/input/rsna-models/last_liver_fold0.pt"))
liverModel_0 = liverModel_0.eval()

liverModel_2  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
liverModel_2.load_state_dict(torch.load("/kaggle/input/rsna-models/liver/liver/liver_fold2.pt"))
liverModel_2 = liverModel_2.eval()
liver_koefs_2 = [0.08499399, 0.05209296, 2.82477031]

liverModel_3  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
liverModel_3.load_state_dict(torch.load("/kaggle/input/rsna-models/liver/liver/liver_fold3.pt"))
liverModel_3 = liverModel_3.eval()
liver_koefs_3 = [0.04162182, 0.01810312, 3.08084416]

liverModel_4  = ClassifierResNet3dCSN2P1D(num_classes=3, encoder='r152ir', pool="max").to(torch.float32).to(DEVICE)
liverModel_4.load_state_dict(torch.load("/kaggle/input/rsna-models/liver/liver/liver_fold4.pt"))
liverModel_4 = liverModel_4.eval()
liver_koefs_4 = [0.5054786, 0.3725675, 2.10124541]

#liver (z, y, x) (64, 224, 224)

In [ ]:
ExtrPrediction  = ExtravasationPrediction(path_to_yolo_weights = "/kaggle/input/extr-models/best_yolo.pt",
                                          path_to_pred_models = "/kaggle/input/extr-models",
                                          device = DEVICE)

In [ ]:
constants = np.array([0.974956, 0.121623,
                      0.931948, 1.454767,
                      0.93764, 0.150843, 0.127324, 
                      0.893683, 0.339724, 0.117822,
                      0.883248, 0.261024, 0.20331, 
                      1.24976168e+00], dtype=np.float64)

In [ ]:
# import torchvision
# classifier2D  = torchvision.models.efficientnet_v2_m()
# classifier2D.classifier[1] = torch.nn.Linear(1280, 3)
# classifier2D.load_state_dict(torch.load("/kaggle/input/rsna-models/effnet2d_state.pth"))
# classifier2D = classifier2D.to(torch.half).to(DEVICE).eval()

In [ ]:
def getTopkScans(image3d, main_axis:int=0, topK:int=5):

    image3d = image3d.squeeze(0)
    index_axis = 3 if main_axis == 1 else 2
    height_sum = torch.count_nonzero(image3d.squeeze(0).mean(0), dim=(0, main_axis))

    desc_values = torch.argsort(height_sum, descending=True)[:topK]
    images = torch.index_select(image3d, index_axis , desc_values)

    return images
def get2DInfer(image3d):
    indicesHeight = getTopkScans(image3d, main_axis = 1, topK = 5)
    indicesHeight = indicesHeight.permute(-1, 0, 1, 2)
    return indicesHeight

In [ ]:
patientsData = []
for patient_id in tqdm(sorted(os.listdir(IMAGES_PATH))):
    per_patient_list = []
    per_patient_list.append(patient_id)
    #bowel

    for paths in [glob.glob(os.path.join(IMAGES_PATH, patient_id, "*"))]:
        path = getBestScanId(paths)

                        
        # для паблик теста, при одном скане получаем NaN
        if len(os.listdir(path)) == 1:
            per_patient_list.extend(constants[:-1])
            continue
        if len(os.listdir(path)) <= 400:
            koeff = 1
        else:
            koeff = int(len(os.listdir(path)) // 200)
            
        try:
            cube = get_cube(path, koeff)
            segmentation_sample = prepare_cube_for_segmentation(cube, 
                                                                width=256, 
                                                                height=256, 
                                                                slice_size=112)['image'].to(torch.float32).to(DEVICE)
#             modelSegm = modelSegm.to(DEVICE)
            with torch.no_grad():
                pred = modelSegm(segmentation_sample)['mask'].argmax(dim=1).detach().cpu().numpy()[0]
#             modelSegm = modelSegm.to('cpu')
            torch.cuda.empty_cache()
            del segmentation_sample

            bounding_boxes = findSingleBoundingBoxPerClass(pred, padding=0.0)
        except:
            per_patient_list.extend(constants[:-1])
            continue
        
        try:
            if 1 in bounding_boxes.keys():
                liver = getOrganCube(cube, bounding_boxes, label=1, padding=0.0)
                processedLiver = getOrganForClassification(liver, 
                                                                    width=224, 
                                                                    height=224, 
                                                                    slice_size=64)['image'].to(torch.float32).to(DEVICE)
                with torch.no_grad():
    #                     liverModel_0 = liverModel_0.to(DEVICE)
                    predLiver_0 = liverModel_0(processedLiver).softmax(1).cpu().numpy()[0]
                    # predLiver_0 *= [0.60987788438797, 0.18113791942596436, 0.20898419618606567]
    #                     liverModel_0 = liverModel_0.to('cpu')

    #                     liverModel_2 = liverModel_2.to(DEVICE)
                    predLiver_2 = liverModel_2(processedLiver).softmax(1).cpu().numpy()[0]
                    # predLiver_2 *= [0.0573028102517128, 0.05544815957546234, 0.8872490525245667]
    #                     liverModel_2 = liverModel_2.to('cpu')

    #                     liverModel_3 = liverModel_3.to(DEVICE)
                    predLiver_3 = liverModel_3(processedLiver).softmax(1).cpu().numpy()[0]
                    # predLiver_3 *= [0.04373353719711304, 0.04271698370575905, 0.9135494828224182]
    #                     liverModel_3 = liverModel_3.to('cpu')

    #                     liverModel_4 = liverModel_4.to(DEVICE)
                    predLiver_4 = liverModel_4(processedLiver).softmax(1).cpu().numpy()[0]
                    # predLiver_4 *= [0.14689353108406067, 0.12861159443855286, 0.7244949340820312]
    #                     liverModel_4 = liverModel_4.to('cpu')

                predLiver = 0.1 * predLiver_0 + 0.2 * predLiver_2 + 0.3 * predLiver_3 + 0.4 * predLiver_4


                torch.cuda.empty_cache()
                del processedLiver
            else:
                predLiver = [0.893683, 0.339724, 0.117822]
        except:
            predLiver = [0.893683, 0.339724, 0.117822]

            
        try:
            if 2 in bounding_boxes.keys():
                spleen = getOrganCube(cube, bounding_boxes, label=2, padding=0.0)
                processedSpleen = getOrganForClassification(spleen, 
                                                                    width=128, 
                                                                    height=128, 
                                                                    slice_size=64)['image'].to(torch.float32).to(DEVICE)
    #                 slices_2d = get2DInfer(processedSpleen).to("cuda")
    #                 with torch.no_grad():
    #                     predSpleen = classifier2D(slices_2d).softmax(1).mean(0).cpu().tolist()


    #                 del slices_2d
                with torch.no_grad():
    #                     spleenModel_0 = spleenModel_0.to(DEVICE)
                    predSpleen_0 = spleenModel_0(processedSpleen).softmax(1).cpu().numpy()[0]
    #                     spleenModel_0 = spleenModel_0.to('cpu')

    #                     spleenModel_1 = spleenModel_1.to(DEVICE)
                    predSpleen_1 = spleenModel_1(processedSpleen).softmax(1).cpu().numpy()[0]
    #                     spleenModel_1 = spleenModel_1.to('cpu')

    #                     spleenModel_2 = spleenModel_2.to(DEVICE)
                    predSpleen_2 = spleenModel_2(processedSpleen).softmax(1).cpu().numpy()[0]
    #                     spleenModel_2 = spleenModel_2.to('cpu')
    
    #                     spleenModel_3 = spleenModel_3.to(DEVICE)
                    predSpleen_3 = spleenModel_3(processedSpleen).softmax(1).cpu().numpy()[0]
    #                     spleenModel_3 = spleenModel_3.to('cpu')

    #                     spleenModel_4 = spleenModel_4.to(DEVICE)
                    predSpleen_4 = spleenModel_4(processedSpleen).softmax(1).cpu().numpy()[0]
    #                     spleenModel_4 = spleenModel_4.to('cpu')
    
                predSpleen = (predSpleen_0 + predSpleen_1 + predSpleen_2 + predSpleen_3 + predSpleen_4) / 5

                # predSpleen *= [4.386, 3.448, 2.336]

                torch.cuda.empty_cache()
                del processedSpleen
            else:
                predSpleen = [0.883248, 0.261024, 0.20331]
        except:
            predSpleen = [0.883248, 0.261024, 0.20331]
            
        try:
            if (3 in bounding_boxes.keys()) and (4 in bounding_boxes.keys()):
                right_kidney = getOrganCube(cube, bounding_boxes, label=3, padding=0.0)
                left_kidney = getOrganCube(cube, bounding_boxes, label=4, padding=0.0)
                kidneys = concatKidneys(left_kidney, right_kidney)
                processedKidneys = getOrganForClassification(kidneys, 
                                                                    width=192, 
                                                                    height=112, 
                                                                    slice_size=64)['image'].to(torch.float32).to(DEVICE)
    #                 kidneyModel = kidneyModel.to(DEVICE)
                with torch.no_grad():
    #                     kidneyModel_1 = kidneyModel_1.to(DEVICE)
                    predKidneys_1 = kidneyModel_1(processedKidneys).softmax(1).cpu().numpy()[0]
    #                     kidneyModel_1 = kidneyModel_1.to('cpu')

    #                     kidneyModel_2 = kidneyModel_2.to(DEVICE)
                    predKidneys_2 = kidneyModel_2(processedKidneys).softmax(1).cpu().numpy()[0]
    #                     kidneyModel_2 = kidneyModel_2.to('cpu')
    
#                     kidneyModel_2 = kidneyModel_2.to(DEVICE)
                    predKidneys_4 = kidneyModel_4(processedKidneys).softmax(1).cpu().numpy()[0]
#                     kidneyModel_2 = kidneyModel_2.to('cpu')

                predKidneys = 0.3 * predKidneys_1 + 0.3 * predKidneys_2 + 0.4 * predKidneys_4

                # predKidneys *= [4.45, 2.38, 1.46]

                torch.cuda.empty_cache()
                del processedKidneys
            else:
                predKidneys = [0.93764, 0.150843, 0.127324]
        except:
            predKidneys = [0.93764, 0.150843, 0.127324]

        pred_extravasation = ExtrPrediction.getExtravasationPrediction(cube, batch_size=50)
        # pred_extravasation *= [2.25, 1]
#         pred_extravasation = [0.931948, 1.454767]
        
        #bowel   
        per_patient_list.append(constants[0])
        per_patient_list.append(constants[1])
        #extravasion
        per_patient_list.extend(pred_extravasation)  
        
        #три значения для органов в порядке из датафрейма
#         predKidneys = [0.93764, 0.150843, 0.127324]
        per_patient_list.extend(predKidneys)
        
#         predLiver = [0.893683, 0.339724, 0.117822]
        per_patient_list.extend(predLiver)
        
#         predSpleen = [0.883248, 0.261024, 0.20331]
        per_patient_list.extend(predSpleen)
        #any_injury
#         per_patient_list.append(constants[-1])
#     print(per_patient_list)
    patientsData.append(per_patient_list)
        # для последующего ресайза органов и подготовки их к классификации можно использовать функцию prepare_cube_for_segmentation с соотв. параметрами
        
        

In [ ]:
# columns = list(columns)
# columns.append('any_injury')
# submission = pd.read_csv('/kaggle/input/rsna-2023-abdominal-trauma-detection/sample_submission.csv')
submission = pd.DataFrame(patientsData, columns=columns)

In [ ]:
# def check_injury(series)->bool:
#     return series.bowel_healthy > series.bowel_healthy

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)